In [3]:
!pip3 install -qqq pydantic==2.5.2
!python3 -c 'import pydantic; print(pydantic.__version__)'

import sys

!{sys.executable} -m pip install -qqq pydantic==2.5.2

2.5.2


In [5]:
import pydantic

pydantic.__version__

'2.5.2'

In [6]:
# !python3 -m -qqq pip install nb_mypy
# %load_ext nb_mypy # Setup mypy extension for type checking.

In [25]:
from datetime import datetime

from pydantic import BaseModel


class Delivery(BaseModel):
    timestamp: datetime
    dimensions: tuple[int, int]

In [8]:
from datetime import datetime
from typing import Tuple

from pydantic import BaseModel


class Delivery(BaseModel):
    timestamp: datetime
    dimensions: Tuple[int, int]

In [9]:
m = Delivery(timestamp="2023-01-02T03:04:05Z", dimensions=["10", "20"])
print(repr(m.timestamp))

datetime.datetime(2023, 1, 2, 3, 4, 5, tzinfo=TzInfo(UTC))


In [10]:
print(m.dimensions)

(10, 20)


## Pydantic Examples

In [27]:
from datetime import datetime

from pydantic import BaseModel, PositiveInt


class User(BaseModel):
    id: int
    name: str = "John Doe"
    signup_ts: datetime | None
    tastes: dict[str, PositiveInt]


external_data = {
    "id": "123",
    "signup_ts": "2019-06-01 12:22",  # ISO8601 str will be converted to datetime
    "tastes": {"wine": 9, b"cheese": 7, "cabbage": "1"},
}

user = User(**external_data)
print(type(user.id))
print(repr(user.signup_ts))
print(user.tastes)

<class 'int'>
datetime.datetime(2019, 6, 1, 12, 22)
{'wine': 9, 'cheese': 7, 'cabbage': 1}


In [12]:
from pydantic import ValidationError

external_data = {
    "id": "not a string",
    "tastes": {},
}
try:
    User(**external_data)
except ValidationError as e:  # Useful tip: ex-e-as-e
    print(e.errors())

[{'type': 'int_parsing', 'loc': ('id',), 'msg': 'Input should be a valid integer, unable to parse string as an integer', 'input': 'not a string', 'url': 'https://errors.pydantic.dev/2.5/v/int_parsing'}, {'type': 'missing', 'loc': ('signup_ts',), 'msg': 'Field required', 'input': {'id': 'not a string', 'tastes': {}}, 'url': 'https://errors.pydantic.dev/2.5/v/missing'}]


In [16]:
import json

User.model_json_schema()

{'properties': {'id': {'title': 'Id', 'type': 'integer'},
  'name': {'default': 'Jane Doe', 'title': 'Name', 'type': 'string'}},
 'required': ['id'],
 'title': 'User',
 'type': 'object'}

In [17]:
from pydantic import BaseModel


class User(BaseModel):
    id: int
    name: str = "Jane Doe"

In [18]:
user = User(id="1")
user

User(id=1, name='Jane Doe')

In [19]:
from datetime import datetime, timezone

from pydantic import BaseModel, field_validator


class Meeting(BaseModel):
    when: datetime

    @field_validator("when", mode="wrap")
    def when_now(cls, input_value, handler):
        if input_value == "now":
            return datetime.now()
        when = handler(input_value)
        # in this specific application we know tz naive datetimes are in UTC
        if when.tzinfo is None:
            when = when.replace(tzinfo=timezone.utc)
        return when

In [21]:
print(Meeting(when="2020-01-01T12:00+01:00"))

when=datetime.datetime(2020, 1, 1, 12, 0, tzinfo=TzInfo(+01:00))


In [22]:
print(Meeting(when="now"))

when=datetime.datetime(2023, 12, 17, 23, 7, 17, 341624)


In [23]:
print(Meeting(when="2020-01-01T12:00"))

when=datetime.datetime(2020, 1, 1, 12, 0, tzinfo=datetime.timezone.utc)


## Annotated Validators

In [32]:
from pydantic import BaseModel, ValidationError
from pydantic.functional_validators import AfterValidator
from typing_extensions import Annotated


def check_email(s: str) -> str:
    assert "@" in s, f"{s} is not an email"
    return s


Email = Annotated[str, AfterValidator(check_email)]


class Account(BaseModel):
    emails: list[Email]

In [33]:
try:
    account = Account(emails=["john@mail.com", "jane"])
except ValidationError as e:
    print(e.errors())

[{'type': 'assertion_error', 'loc': ('emails', 1), 'msg': 'Assertion failed, jane is not an email', 'input': 'jane', 'ctx': {'error': AssertionError('jane is not an email')}, 'url': 'https://errors.pydantic.dev/2.5/v/assertion_error'}]


In [34]:
Account(emails=["john@mail.com", "jane@mail.com"])

Account(emails=['john@mail.com', 'jane@mail.com'])